In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
#from google.colab import files
#data_to_load = files.upload()

In [3]:
#df = pd.read_csv("df_energy_weather.csv", index_col=[0])

In [4]:
df = pd.read_csv("../raw_data/df_energy_weather.csv", index_col=[0])
#print(df.columns)

#pd.options.display.max_columns
pd.set_option("display.max_columns", None)
df.columns

Index(['Date', 'Code INSEE région', 'Consommation (MW)', 'Thermique (MW)',
       'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)',
       'Pompage (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'Stockage batterie', 'Déstockage batterie', 'Eolien terrestre',
       'Eolien offshore', 'TCO Thermique (%)', 'TCH Thermique (%)',
       'TCO Nucléaire (%)', 'TCH Nucléaire (%)', 'TCO Eolien (%)',
       'TCH Eolien (%)', 'TCO Solaire (%)', 'TCH Solaire (%)', 'Column 30',
       'YEAR', 'MONTH', 'DAY', 'T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE',
       'T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M',
       'WS50M', 'sin_month', 'cos_month', 'week_day', 'sin_day', 'cos_day',
       'bank_holiday', 'school_holidays'],
      dtype='object')

In [5]:
df = df[['Consommation (MW)', 'T2M', 'TS', 'week_day','sin_month', 'cos_month','sin_day', 'cos_day','bank_holiday', 'school_holidays']]
df.dtypes

Consommation (MW)    float64
T2M                  float64
TS                   float64
week_day               int64
sin_month            float64
cos_month            float64
sin_day              float64
cos_day              float64
bank_holiday         float64
school_holidays        int64
dtype: object

In [6]:
len_ = int(0.8*df.shape[0])
df_train = df[:len_]
#df_train = df[:1014]
df_test = df[len_:]
df_train

,Consommation (MW),T2M,TS,week_day,sin_month,cos_month,sin_day,cos_day,bank_holiday,school_holidays
0,252371.0,7.30,7.08,1,5.000000e-01,0.866025,0.781831,0.623490,1.0,1
1,280064.0,4.36,4.10,2,5.000000e-01,0.866025,0.974928,-0.222521,0.0,1
2,285526.0,5.46,3.78,3,5.000000e-01,0.866025,0.433884,-0.900969,0.0,1
3,286799.0,5.92,4.06,4,5.000000e-01,0.866025,-0.433884,-0.900969,0.0,1
4,267848.0,6.64,4.85,5,5.000000e-01,0.866025,-0.974928,-0.222521,0.0,1
...,...,...,...,...,...,...,...,...,...,...
2728,166236.0,21.52,22.05,6,1.224647e-16,-1.000000,-0.781831,0.623490,0.0,0
2729,182157.0,24.25,25.48,0,1.224647e-16,-1.000000,0.000000,1.000000,0.0,0
2730,188411.0,24.76,25.70,1,1.224647e-16,-1.000000,0.781831,0.623490,0.0,0
2731,189295.0,24.42,25.37,2,1.224647e-16,-1.000000,0.974928,-0.222521,0.0,0


In [7]:
from sklearn.preprocessing import MinMaxScaler
columns = df.columns

scaler = MinMaxScaler()
df_train = pd.DataFrame(scaler.fit_transform(df_train), columns=columns)
df_test = pd.DataFrame(scaler.transform(df_test), columns=columns)


In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers, metrics
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
def train_model(input_shape_value):
    metric = metrics.MAPE

    

    model = models.Sequential()
    #model.add(layers.Masking(mask_value=-1000., input_shape=(30,2)))
    #model.add(layers.LSTM(20, return_sequences=True, activation='tanh'))
    # Ajout d'une couche + drop


    model.add(layers.LSTM(20, return_sequences = True, input_shape=input_shape_value, activation='tanh'))
    model.add(layers.Dropout(0.2))
    #model.add(layers.LSTM(30, return_sequences = True, activation='tanh'))
    #model.add(layers.Dropout(0.2))
    #model.add(layers.LSTM(20, return_sequences = True, activation='tanh'))
    #model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(10, return_sequences = False, activation='tanh'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(5, activation = 'relu'))
    model.add(layers.Dense(14, activation = 'linear'))


    model.compile(loss='mse', 
                  optimizer=RMSprop(learning_rate=0.0003), 
                  metrics=[metric])
    
    

    return model


In [10]:
def grid(df_train,df_test,nb_jours_entrainement):
    steps = 1
    #nb_jours_entrainement = 360
    nb_jours_prevision = 14
    df_train_shape = df_train.shape[0]
    df_test_shape = df_test.shape[0]
    
    X_train = []
    y_train = []

    for i in range(nb_jours_entrainement, df_train_shape - nb_jours_prevision, steps):
        X_train.append(df_train[i-nb_jours_entrainement:i])
        #y_test.append(df_train[i : i + nb_jours_prevision,0:1])
        #.iloc[:, 1:2]
        y_train.append(df_train.iloc[i : i + nb_jours_prevision, 0:1])
        
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    input_shape_value = X_train.shape[1:3]
    
    model = train_model(input_shape_value)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
 
    model.fit(X_train, y_train,
            validation_split=0.2,
            epochs=1000, 
            batch_size=32,
            callbacks=[es], verbose=1)
    
    X_test = []
    y_test = []

    for i in range(nb_jours_entrainement, df_test_shape - nb_jours_prevision, steps):
        X_test.append(df_test[i-nb_jours_entrainement:i])
        #y_test.append(df_train[i : i + nb_jours_prevision,0:1])
        #.iloc[:, 1:2]
        y_test.append(df_test.iloc[i : i + nb_jours_prevision, 0:1])
    
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    res = model.evaluate(X_test, y_test, verbose=1)
    
    return res

In [11]:
list_nb_jours_entrainement = [360]#[200,300,365,420,510]
nombre_couche = 4
nombre_features = 10
infos_sup = 'drop de 0,2'
list_result = []

In [12]:
for nb_jours_entrainement in list_nb_jours_entrainement:
    res = grid(df_train,df_test,nb_jours_entrainement)
    result = f'Nb lstm = {nombre_couche} // Nb features = {nombre_features} // Tl sequence = {nb_jours_entrainement} // LOSS = {res[0]:.5f} // MAPE = {res[1]:.5f} // Infos sup = {infos_sup}'
    print(result)
    list_result.append(result)

2022-06-19 18:39:51.116218: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-19 18:39:51.354971: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
59/59 [==============================] - 8s 105ms/step - loss: 0.1467 - mean_absolute_percentage_error: 93.2308 - val_loss: 0.0852 - val_mean_absolute_percentage_error: 116545.0078
Epoch 2/1000
59/59 [==============================] - 6s 100ms/step - loss: 0.1243 - mean_absolute_percentage_error: 80.3516 - val_loss: 0.0733 - val_mean_absolute_percentage_error: 156482.7969
Epoch 3/1000
59/59 [==============================] - 6s 100ms/step - loss: 0.1076 - mean_absolute_percentage_error: 71.3648 - val_loss: 0.0622 - val_mean_absolute_percentage_error: 162130.5469
Epoch 4/1000
59/59 [==============================] - 6s 99ms/step - loss: 0.0926 - mean_absolute_percentage_error: 63.0889 - val_loss: 0.0523 - val_mean_absolute_percentage_error: 200070.5781
Epoch 5/1000
59/59 [==============================] - 6s 102ms/step - loss: 0.0789 - mean_absolute_percentage_error: 55.4982 - val_loss: 0.0438 - val_mean_absolute_percentage_error: 240952.0625
Epoch 6/1000
59/59 [===========

In [13]:
list_result

['Nb lstm = 4 // Nb features = 10 // Tl sequence = 360 // LOSS = 0.00810 // MAPE = 20.90526 // Infos sup = drop de 0,2']